In [ ]:
!pip install neo4j
!pip install unstructured
!pip install "unstructured[pdf]"

  Preparing metadata (setup.py) ... done
  Using cached contourpy-1.3.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.8-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (6.2 kB)
  Using cached pyparsing-3.2.1-py3-none-any.whl.metadata (5.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 96.2 kB/s eta 0:00:0000:0100:07
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 47.6 kB/s eta 0:00:0000:0300:08m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.4 kB/s eta 0:00:00a 0:00:04
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 57.4 kB/s eta 0:00:00a 0:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 273.9 kB/s eta 0:00:0000:0100:07
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 70.3 kB/s eta 0:00:0000:0300:06m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 101.1 kB/s eta 0:00:0000:010:0

In [2]:
!pip install --user -U nltk
!python -m nltk.downloader popular

/home/teng/miniconda/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/teng/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /home/teng/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /home/teng/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /home/teng/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /home/teng/nltk_data...
[nltk_d

In [4]:
!sudo apt-get install poppler-utils

[sudo] password for teng: 
sudo: a password is required
^C


In [ ]:
!sudo apt-get install tesseract-ocr

In [ ]:
!pip install tesseract

In [1]:
import os
from neo4j import GraphDatabase
import uuid
import hashlib
from dotenv import load_dotenv

load_dotenv()

# Neo4j graph database can be self-hosted or local.
# Or go to https://neo4j.com/cloud/platform/aura-graph-database/ to create a FREE AuraDB instance.
# Fill conection details below
NEO4J_URL = os.getenv("NEO4J_URI")
NEO4J_USER = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
NEO4J_DATABASE = "neo4j"

In [ ]:
# def initialiseNeo4j():
#     cypher_schema = [
#         "CREATE CONSTRAINT sectionKey IF NOT EXISTS FOR (c:Section) REQUIRE (c.key) IS UNIQUE;",
#         "CREATE CONSTRAINT chunkKey IF NOT EXISTS FOR (c:Chunk) REQUIRE (c.key) IS UNIQUE;",
#         "CREATE CONSTRAINT documentKey IF NOT EXISTS FOR (c:Document) REQUIRE (c.url_hash) IS UNIQUE;",
#         "CREATE CONSTRAINT tableKey IF NOT EXISTS FOR (c:Table) REQUIRE (c.key) IS UNIQUE;",
#         "CREATE CONSTRAINT elementKey IF NOT EXISTS FOR (c:Element) REQUIRE (c.key) IS UNIQUE;",
#         "CALL db.index.vector.createNodeIndex('chunkVectorIndex', 'Embedding', 'value', 1536, 'COSINE');"
#     ]

#     driver = GraphDatabase.driver(NEO4J_URL, database=NEO4J_DATABASE, auth=(NEO4J_USER, NEO4J_PASSWORD))

#     with driver.session() as session:
#         for cypher in cypher_schema:
#             session.run(cypher)
#     driver.close()

# def ingestDocumentNeo4j(elements, doc_location):

#     cypher_pool = [
#         # 0 - Document
#         "MERGE (d:Document {url_hash: $doc_url_hash_val}) ON CREATE SET d.url = $doc_url_val, d.last_modified = $doc_last_modified_val RETURN d;",
#         # 1 - Section
#         "MERGE (p:Section {key: $element_id_val}) ON CREATE SET p:Element, p.page_idx = $page_idx_val, p.title_hash = $title_hash_val, p.block_idx = $block_idx_val, p.title = $title_val, p.tag = $tag_val RETURN p;",
#         # 2 - Link Section with the Document
#         "MATCH (d:Document {url_hash: $doc_url_hash_val}) MATCH (s:Section {key: $element_id_val}) MERGE (d)<-[:HAS_DOCUMENT]-(s);",
#         # 3 - Link Section with a parent Element
#         "MATCH (s1:Section {key: $element_id_val}) MATCH (s2:Element {key: $sec_parent_element_id_val}) MERGE (s2)<-[:UNDER_SECTION]-(s1);",
#         # 4 - Chunk
#         "MERGE (c:Chunk {key: $element_id_val}) ON CREATE SET c:Element, c.sentences = $sentences_val, c.sentences_hash = $sentences_hash_val, c.block_idx = $block_idx_val, c.page_idx = $page_idx_val, c.tag = $tag_val RETURN c;",
#         # 5 - Link Chunk to another element
#         "MATCH (c:Chunk {key: $element_id_val}) MATCH (s:Element {key:$chk_parent_element_id_val}) MERGE (s)<-[:HAS_PARENT]-(c);",
#         # 6 - Table
#         "MERGE (t:Table {key: $element_id_val}) ON CREATE SET t:Element, t.name = $name_val, t.doc_url_hash = $doc_url_hash_val, t.block_idx = $block_idx_val, t.page_idx = $page_idx_val, t.html = $html_val, t.rows = $rows_val RETURN t;",
#         # 7 - Link Table to Section
#         "MATCH (t:Table {key: $element_id_val}) MATCH (s:Section {key: $tb_parent_element_id_val}) MERGE (s)<-[:HAS_PARENT]-(t);",
#         # 8 - Link Table to Document
#         "MATCH (t:Table {key: $element_id_val}) MATCH (s:Document {url_hash: $doc_url_hash_val}) MERGE (s)<-[:HAS_PARENT]-(t);",
#         # 9 - Image
#         "MERGE (t:Image {key: $element_id_val}) ON CREATE SET t:Element, t.name = $name_val, t.doc_url_hash = $doc_url_hash_val, t.block_idx = $block_idx_val, t.page_idx = $page_idx_val RETURN t;",
#         # 10 - Link Image to Document
#         "MATCH (t:Image {key: $element_id_val}) MATCH (s:Document {url_hash: $doc_url_hash_val}) MERGE (s)<-[:HAS_PARENT]-(t);",
#         # 11 - Link top Chunk to Document
#         "MATCH (t:Chunk {key: $element_id_val}) MATCH (s:Document {url_hash: $doc_url_hash_val}) MERGE (s)<-[:HAS_PARENT]-(t);"
#     ]

#     driver = GraphDatabase.driver(NEO4J_URL, database=NEO4J_DATABASE, auth=(NEO4J_USER, NEO4J_PASSWORD))

#     with driver.session() as session:
#         cypher = ""

#         # 1 - Create Document node
#         doc_url_val = doc_location
#         doc_url_hash_val = hashlib.md5(doc_url_val.encode("utf-8")).hexdigest()
#         doc_last_modified_val = elements[0].metadata.last_modified

#         cypher = cypher_pool[0]
#         session.run(cypher, doc_url_hash_val=doc_url_hash_val, doc_url_val=doc_url_val, doc_last_modified_val=doc_last_modified_val)

#         # 2 - Create Section nodes if element.category = 'Title'

#         countSection = 0
#         countChunk = 0
#         countTable = 0
#         countImage = 0

#         # iterate all items in list elements and keep an index i
#         for i, sec in enumerate(elements) :

#             tag_val = sec.category
#             page_idx_val = sec.metadata.page_number
#             block_idx_val = i
#             element_id_val = sec.id
#             text_val = sec.text
#             text_hash_val = hashlib.md5(text_val.encode("utf-8")).hexdigest()
#             parent_id_val = str(sec.metadata.parent_id)

#             if sec.category == 'Title':

#                 # MERGE section node
#                 cypher = cypher_pool[1]
#                 session.run(cypher, page_idx_val=page_idx_val
#                                     , title_hash_val=text_hash_val
#                                     , title_val=text_val
#                                     , tag_val=tag_val
#                                     , block_idx_val=block_idx_val
#                                     , doc_url_hash_val=doc_url_hash_val
#                                     , element_id_val=element_id_val
#                         )

#                 # Link Section with a parent section or Document

#                 if parent_id_val == "None":    # use Document as parent
#                     cypher = cypher_pool[2]
#                     session.run(cypher
#                                         , doc_url_hash_val=doc_url_hash_val
#                                         , element_id_val=element_id_val
#                         )

#                 else:   # use parent section
#                     cypher = cypher_pool[3]
#                     session.run(cypher
#                                         , sec_parent_element_id_val=parent_id_val
#                                         , doc_url_hash_val=doc_url_hash_val
#                                         , element_id_val=element_id_val
#                                 )
#                 # **** if sec_parent_val == "None":

#                 countSection += 1
#                 continue
#             # **** for sec in elements: category = 'Title'


#         # ------- Continue within the session block -------
#         # 3 - Create Chunk nodes from chunks

#             if sec.category == 'NarrativeText' or sec.category == 'List' or sec.category == 'ListItem' \
#                 or sec.category == 'UncategorizedText' or sec.category == 'Header':


#                 # MERGE chunk node
#                 cypher = cypher_pool[4]
#                 session.run(cypher, sentences_hash_val=text_hash_val
#                                     , sentences_val=text_val
#                                     , block_idx_val=block_idx_val
#                                     , page_idx_val=page_idx_val
#                                     , tag_val=tag_val
#                                     , doc_url_hash_val=doc_url_hash_val
#                                     , element_id_val=element_id_val
#                             )

#                 # Link chunk with a parent Element. If none, link it to Document

#                 if not parent_id_val == "None":

#                     cypher = cypher_pool[5]
#                     session.run(cypher
#                                     , doc_url_hash_val=doc_url_hash_val
#                                     , chk_parent_element_id_val=parent_id_val
#                                     , element_id_val=element_id_val
#                                 )
#                 else:   # link chunk to Document
#                     cypher = cypher_pool[11]
#                     session.run(cypher
#                                     , doc_url_hash_val=doc_url_hash_val
#                                     , element_id_val=element_id_val
#                                 )

#                 countChunk += 1
#                 continue
#             # **** for sec in elements: Chunk

#             # 4 - Create Table nodes

#             if sec.category == 'Table':

#                 html_val = sec.metadata.text_as_html
#                 # count <tr> in html
#                 rows_val = len(html_val.split('</tr>'))

#                 # MERGE table node

#                 cypher = cypher_pool[6]
#                 session.run(cypher, block_idx_val=block_idx_val
#                                 , page_idx_val=page_idx_val
#                                 , name_val=text_val
#                                 , html_val=html_val
#                                 , rows_val=rows_val
#                                 , doc_url_hash_val=doc_url_hash_val
#                                 , element_id_val=element_id_val
#                             )

#                 # Link table with a section
#                 # Table always has a parent section

#                 if not parent_id_val == "None":
#                     cypher = cypher_pool[7]
#                     session.run(cypher
#                                     , tb_parent_element_id_val=parent_id_val
#                                     , element_id_val=element_id_val
#                                 )

#                 else:   # link table to Document
#                     cypher = cypher_pool[8]
#                     session.run(cypher
#                                     , doc_url_hash_val=doc_url_hash_val
#                                     , element_id_val=element_id_val
#                                 )
#                 countTable += 1
#                 continue
#             # **** for sec in elements: category = 'Table'


#         # 5 - Create Image nodes

#             if sec.category == 'Image':

#                 # MERGE Image node

#                 cypher = cypher_pool[9]
#                 session.run(cypher, block_idx_val=block_idx_val
#                                 , page_idx_val=page_idx_val
#                                 , name_val=text_val
#                                 , doc_url_hash_val=doc_url_hash_val
#                                 , element_id_val=element_id_val
#                             )

#                 # Link image with a section
#                 # Image always linkes to Document

#                 cypher = cypher_pool[10]
#                 session.run(cypher
#                                 , image_parent_element_id_val=doc_url_hash_val
#                                 , element_id_val=element_id_val
#                                 , doc_url_hash_val=doc_url_hash_val
#                             )

#                 countImage += 1
#                 continue
#             # **** for sec in elements: category = 'Image'
#         # *** for i, sec in enumerate(elements) :

#         print(f'\'{doc_url_val}\' Done! Summary: ')
#         print('#Sections: ' + str(countSection))
#         print('#Chunks: ' + str(countChunk))
#         print('#Tables: ' + str(countTable))
#         print('#Images: ' + str(countImage))

#     # *** with driver.session() as session:

#     driver.close()

In [2]:
from neo4j import GraphDatabase
import hashlib

# Constants for Element Categories
TITLE = "Title"
TEXT_CATEGORIES = {"NarrativeText", "List", "ListItem", "UncategorizedText", "Header"}
TABLE = "Table"
IMAGE = "Image"

def initialise_neo4j():
    """Initializes Neo4j constraints and vector index."""
    cypher_schema = [
        "CREATE CONSTRAINT sectionKey IF NOT EXISTS FOR (c:Section) REQUIRE (c.key) IS UNIQUE;",
        "CREATE CONSTRAINT chunkKey IF NOT EXISTS FOR (c:Chunk) REQUIRE (c.key) IS UNIQUE;",
        "CREATE CONSTRAINT documentKey IF NOT EXISTS FOR (c:Document) REQUIRE (c.url_hash) IS UNIQUE;",
        "CREATE CONSTRAINT tableKey IF NOT EXISTS FOR (c:Table) REQUIRE (c.key) IS UNIQUE;",
        "CREATE CONSTRAINT elementKey IF NOT EXISTS FOR (c:Element) REQUIRE (c.key) IS UNIQUE;",
        "CALL db.index.vector.createNodeIndex('chunkVectorIndex', 'Embedding', 'value', 1536, 'COSINE');"
    ]

    with GraphDatabase.driver(NEO4J_URL, database=NEO4J_DATABASE, auth=(NEO4J_USER, NEO4J_PASSWORD)) as driver:
        with driver.session() as session:
            for cypher in cypher_schema:
                session.run(cypher)

def execute_cypher(session, query, **params):
    """Helper function to execute a Cypher query with parameters."""
    session.run(query, **params)

def ingest_document_neo4j(elements, doc_location):
    """Ingests a document and its elements into Neo4j."""
    cypher_queries = {
        "document": "MERGE (d:Document {url_hash: $doc_url_hash}) ON CREATE SET d.url = $doc_url, d.last_modified = $doc_last_modified RETURN d;",
        "section": "MERGE (p:Section {key: $element_id}) ON CREATE SET p:Element, p.page_idx = $page_idx, p.title_hash = $title_hash, p.block_idx = $block_idx, p.title = $title, p.tag = $tag RETURN p;",
        "link_section_document": "MATCH (d:Document {url_hash: $doc_url_hash}) MATCH (s:Section {key: $element_id}) MERGE (d)<-[:HAS_DOCUMENT]-(s);",
        "link_section_parent": "MATCH (s1:Section {key: $element_id}) MATCH (s2:Element {key: $parent_element_id}) MERGE (s2)<-[:UNDER_SECTION]-(s1);",
        "chunk": "MERGE (c:Chunk {key: $element_id}) ON CREATE SET c:Element, c.sentences = $sentences, c.sentences_hash = $sentences_hash, c.block_idx = $block_idx, c.page_idx = $page_idx, c.tag = $tag RETURN c;",
        "link_chunk_parent": "MATCH (c:Chunk {key: $element_id}) MATCH (s:Element {key: $parent_element_id}) MERGE (s)<-[:HAS_PARENT]-(c);",
        "link_chunk_document": "MATCH (c:Chunk {key: $element_id}) MATCH (s:Document {url_hash: $doc_url_hash}) MERGE (s)<-[:HAS_PARENT]-(c);",
        "table": "MERGE (t:Table {key: $element_id}) ON CREATE SET t:Element, t.name = $name, t.doc_url_hash = $doc_url_hash, t.block_idx = $block_idx, t.page_idx = $page_idx, t.html = $html, t.rows = $rows RETURN t;",
        "link_table_section": "MATCH (t:Table {key: $element_id}) MATCH (s:Section {key: $parent_element_id}) MERGE (s)<-[:HAS_PARENT]-(t);",
        "link_table_document": "MATCH (t:Table {key: $element_id}) MATCH (s:Document {url_hash: $doc_url_hash}) MERGE (s)<-[:HAS_PARENT]-(t);",
        "image": "MERGE (i:Image {key: $element_id}) ON CREATE SET i:Element, i.name = $name, i.doc_url_hash = $doc_url_hash, i.block_idx = $block_idx, i.page_idx = $page_idx RETURN i;",
        "link_image_document": "MATCH (i:Image {key: $element_id}) MATCH (s:Document {url_hash: $doc_url_hash}) MERGE (s)<-[:HAS_PARENT]-(i);"
    }

    doc_url_hash = hashlib.md5(doc_location.encode("utf-8")).hexdigest()
    doc_last_modified = elements[0].metadata.last_modified

    with GraphDatabase.driver(NEO4J_URL, database=NEO4J_DATABASE, auth=(NEO4J_USER, NEO4J_PASSWORD)) as driver:
        with driver.session() as session:
            # Insert document node
            execute_cypher(session, cypher_queries["document"], doc_url_hash=doc_url_hash, doc_url=doc_location, doc_last_modified=doc_last_modified)

            for i, sec in enumerate(elements):
                element_id = sec.id
                parent_id = str(sec.metadata.parent_id)
                tag = sec.category
                page_idx = sec.metadata.page_number
                block_idx = i
                text = sec.text
                text_hash = hashlib.md5(text.encode("utf-8")).hexdigest()

                if tag == TITLE:
                    # Insert Section node
                    execute_cypher(session, cypher_queries["section"], element_id=element_id, page_idx=page_idx, title_hash=text_hash, title=text, tag=tag, block_idx=block_idx)

                    # Link Section to Document or Parent
                    if parent_id == "None":
                        execute_cypher(session, cypher_queries["link_section_document"], doc_url_hash=doc_url_hash, element_id=element_id)
                    else:
                        execute_cypher(session, cypher_queries["link_section_parent"], parent_element_id=parent_id, element_id=element_id)
                    continue

                if tag in TEXT_CATEGORIES:
                    # Insert Chunk node
                    execute_cypher(session, cypher_queries["chunk"], element_id=element_id, sentences=text, sentences_hash=text_hash, block_idx=block_idx, page_idx=page_idx, tag=tag)

                    # Link Chunk to Parent or Document
                    if parent_id != "None":
                        execute_cypher(session, cypher_queries["link_chunk_parent"], parent_element_id=parent_id, element_id=element_id)
                    else:
                        execute_cypher(session, cypher_queries["link_chunk_document"], doc_url_hash=doc_url_hash, element_id=element_id)
                    continue

                if tag == TABLE:
                    rows = len(sec.metadata.text_as_html.split('</tr>'))
                    execute_cypher(session, cypher_queries["table"], element_id=element_id, name=text, html=sec.metadata.text_as_html, rows=rows, doc_url_hash=doc_url_hash, block_idx=block_idx, page_idx=page_idx)

                    if parent_id != "None":
                        execute_cypher(session, cypher_queries["link_table_section"], parent_element_id=parent_id, element_id=element_id)
                    else:
                        execute_cypher(session, cypher_queries["link_table_document"], doc_url_hash=doc_url_hash, element_id=element_id)
                    continue

                if tag == IMAGE:
                    execute_cypher(session, cypher_queries["image"], element_id=element_id, name=text, doc_url_hash=doc_url_hash, block_idx=block_idx, page_idx=page_idx)
                    execute_cypher(session, cypher_queries["link_image_document"], doc_url_hash=doc_url_hash, element_id=element_id)


In [4]:
# create constraints and indexes. only need to execute once.

# initialiseNeo4j()

initialise_neo4j()

SessionExpired: Failed to obtain connection towards 'WRITE' server.

In [4]:
# from unstructured.partition.pdf import partition_pdf
# from unstructured.staging.base import convert_to_dict
# from unstructured.staging.base import elements_to_json


# doc_location = "./data/Agroforestry"   # replace this to your document location
# doc_file_name = "agrof_health_paper.pdf" # replace this to your document file name
# doc_url = doc_location + "/" + doc_file_name

# # partition the pdf into elements

# elements = partition_pdf(
#     filename=doc_url,
#     infer_table_structure=True
# )

# # elements = partition_pdf(
# #     doc_location+"/"+doc_file_name,
# #     doc_location,
# #     output_format="json"
# # )

# ingestDocumentNeo4j(elements, doc_url)

In [20]:
import os
from unstructured.partition.pdf import partition_pdf
from unstructured.staging.base import convert_to_dict, elements_to_json

# Define document directory
doc_location = "./data/assessment"  # Replace with your document location

# Get all PDF files in the directory
pdf_files = [f for f in os.listdir(doc_location) if f.endswith(".pdf")]

for doc_file_name in pdf_files:
    doc_url = os.path.join(doc_location, doc_file_name)
    
    # Partition the PDF into elements
    elements = partition_pdf(filename=doc_url, infer_table_structure=True)

    # seen_keys = set()
    # filtered_elements = []

    # for element in elements:
    #     if element.key not in seen_keys:
    #         seen_keys.add(element.key)
    #         filtered_elements.append(element)

    ingest_document_neo4j(elements, doc_url)

    
    # Ingest the extracted elements into Neo4j
    # ingestDocumentNeo4j(elements, doc_url)
    
    # Save elements as JSON
    json_filename = doc_url + ".json"
    elements_to_json(elements, filename=json_filename)

    print(f"Processed and ingested: {doc_file_name}")


Processed and ingested: uruguay_sslb_framework__2.pdf
Processed and ingested: GCTWF M&E Plan_August 2020.pdf
Processed and ingested: 240126-Tanga_Watershed_Invesment_Program-Business_Case._detailed_presentation Autosaved (003).pdf


In [ ]:
# import os
# from unstructured.partition.pdf import partition_pdf

# # Function to process and ingest each document
# def process_and_ingest(directory):
#     for filename in os.listdir(directory):
#         if filename.endswith(".pdf"):  # Process only PDF files
#             doc_url = os.path.join(directory, filename)
#             print(f"Processing: {filename}")

#             # Partition the PDF into elements
#             elements = partition_pdf(filename=doc_url, infer_table_structure=True)

#             # Ingest extracted elements into Neo4j
#             ingestDocumentNeo4j(elements, doc_url)

# # Directory containing the PDF files
# doc_location = "example-docs"  # Update this with your directory path

# # Process and ingest all PDFs in the directory (Neo4j is already initialized)
# process_and_ingest(doc_location)


In [7]:
# save the elements as a json file

convert_to_dict(elements)

filename = doc_location+"/"+doc_file_name+".json"
elements_to_json(elements, filename=filename)

'[\n    {\n        "element_id": "490bb613217d8aa06a73d158d5599ff9",\n        "metadata": {\n            "coordinates": {\n                "layout_height": 2221,\n                "layout_width": 1654,\n                "points": [\n                    [\n                        100.1,\n                        337.9\n                    ],\n                    [\n                        100.1,\n                        510.6\n                    ],\n                    [\n                        1352.2,\n                        510.6\n                    ],\n                    [\n                        1352.2,\n                        337.9\n                    ]\n                ],\n                "system": "PixelSpace"\n            },\n            "detection_class_prob": 0.69804,\n            "file_directory": "./data/Agroforestry",\n            "filename": "agrof_health_paper.pdf",\n            "filetype": "application/pdf",\n            "languages": [\n                "eng"\n      

In [ ]:
# import os
# from unstructured.partition.pdf import partition_pdf
# from unstructured.staging.base import convert_to_dict, elements_to_json

# # Function to process, ingest, and save elements as JSON
# def process_and_save(directory):
#     for filename in os.listdir(directory):
#         if filename.endswith(".pdf"):  # Process only PDF files
#             doc_url = os.path.join(directory, filename)
#             print(f"Processing: {filename}")

#             # Partition the PDF into elements
#             elements = partition_pdf(filename=doc_url, infer_table_structure=True)

#             # Ingest extracted elements into Neo4j
#             ingestDocumentNeo4j(elements, doc_url)

#             # Save extracted elements as JSON
#             json_filename = os.path.join(directory, filename + ".json")
#             elements_to_json(elements, filename=json_filename)
#             print(f"Saved JSON: {json_filename}")

# # Directory containing the PDF files
# doc_location = "example-docs"  # Update this with your directory path

# # Process, ingest, and save all PDFs in the directory
# process_and_save(doc_location)


In [ ]:
%pip install neo4j sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [8]:
from sentence_transformers import SentenceTransformer
from neo4j import GraphDatabase
import numpy as np

# Load the embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")  # Lightweight & effective

# Neo4j connection (Modify with your credentials)
NEO4J_URL = os.getenv("NEO4J_URI")
NEO4J_USER = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
NEO4J_DATABASE = "neo4j"

driver = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USER, NEO4J_PASSWORD))

def generate_embedding(text):
    """Generate a vector embedding for a given text chunk."""
    return embedding_model.encode(text).tolist()

def ingestDocumentNeo4j(elements, doc_url):
    """Ingest extracted document elements into Neo4j with embeddings."""
    with driver.session() as session:
        for element in elements:
            text_chunk = element.text
            embedding = generate_embedding(text_chunk)

            session.run(
                """
                MERGE (doc:Document {url: $url})
                CREATE (chunk:TextChunk {
                    text: $text,
                    embedding: $embedding
                })
                MERGE (doc)-[:HAS_CHUNK]->(chunk)
                """,
                url=doc_url,
                text=text_chunk,
                embedding=embedding
            )

# Example usage
# doc_location = "example-docs"
# doc_file_name = "layout-parser-paper.pdf"
# doc_url = f"{doc_location}/{doc_file_name}"

# elements = partition_pdf(filename=doc_url, infer_table_structure=True)
# ingestDocumentNeo4j(elements, doc_url)



# Define document directory
# doc_location = "~/development/projects/public/efh-hackathon/notebooks/data/Agroforestry"  # Replace with your document location

doc_location = "/home/teng/development/projects/public/efh-hackathon/notebooks/data/Agroforestry"

# Get all PDF files in the directory
pdf_files = [f for f in os.listdir(doc_location) if f.endswith(".pdf")]

for doc_file_name in pdf_files:
    doc_url = os.path.join(doc_location, doc_file_name)
    
    # Partition the PDF into elements
    elements = partition_pdf(filename=doc_url, infer_table_structure=True)
    
    # Ingest the extracted elements into Neo4j
    ingestDocumentNeo4j(elements, doc_url)

In [9]:
def retrieve_context(query_text, top_k=3):
    """Retrieve top-k relevant document chunks using cosine similarity search in Neo4j."""
    query_embedding = generate_embedding(query_text)  # Get embedding for user query

    with driver.session() as session:
        result = session.run(
            """
            MATCH (chunk:TextChunk)
            RETURN chunk.text AS text, chunk.embedding AS embedding
            """,
        )

        # Compute similarity manually (since Neo4j doesn't natively support vector search)
        docs = []
        for record in result:
            stored_embedding = np.array(record["embedding"])
            similarity = np.dot(stored_embedding, query_embedding) / (
                np.linalg.norm(stored_embedding) * np.linalg.norm(query_embedding)
            )
            docs.append((record["text"], similarity))

        # Sort by similarity and return top_k matches
        sorted_docs = sorted(docs, key=lambda x: x[1], reverse=True)[:top_k]
        return [doc[0] for doc in sorted_docs]


In [10]:
# Example usage
query = "What is the difference between the impact of trees in croplands and PES on food security?"
retrieved_context = retrieve_context(query)
print("Relevant Context:", retrieved_context)

Relevant Context: ['Trees in croplands', 'Trees in croplands', 'Trees in croplands']


In [13]:
import getpass

# Retrieve OpenAI API key securely
# OPENAI_API_KEY = getpass.getpass("Enter your OpenAI API key: ")
TOGETHERAI_API_KEY = getpass.getpass("Enter your TogetherAI API key: ")

In [ ]:
# import openai

# # Set up OpenAI API key
# # client = openai.OpenAI(api_key=OPENAI_API_KEY)

# def generate_rag_response(query):
#     """Retrieve relevant context from Neo4j and use it for LLM response generation."""
#     context = retrieve_context(query)
#     context_text = "\n".join(context)

#     prompt = f"""
#     You are an AI assistant that provides accurate answers using the retrieved context.
#     Try to answer the following question based on the context provided.
#     Do not hallucinate! Only use the provided context for answering.
#     Context:
#     {context_text}

#     Question: {query}
#     Answer:
#     """

#     response = client.chat.completions.create(
#         model="gpt-4o-mini",
#         messages=[
#             {"role": "system", "content": "You are a helpful AI assistant."},
#             {"role": "user", "content": prompt},
#         ]
#     )

#     return response.choices[0].message.content

# # Example query
# query = "How do different tree intervention practices impact total income?"
# response = generate_rag_response(query)
# print("AI Response:", response)


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************pr0A. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [22]:
import openai

# Set up OpenAI API key
# client = openai.OpenAI(api_key=OPENAI_API_KEY)
client = openai.OpenAI(
    api_key=TOGETHERAI_API_KEY,
    base_url="https://api.together.xyz/v1"
)

def generate_rag_response(query):
    """Retrieve relevant context from Neo4j and use it for LLM response generation."""
    context = retrieve_context(query)
    context_text = "\n".join(context)

    prompt = f"""
    You are an AI assistant that provides accurate answers using the retrieved context.
    Try to answer the following question based on the context provided.
    Do not hallucinate! Only use the provided context for answering.
    Please qualify your answer with scientific evidence including effect sizes where available. 
    Don’t be generic in your response but rather be as specific about the Tanga Watershed Investment Program as possible.
    Context:
    {context_text}
    
    Question: {query}
    
    Answer:
    """

    # Question{question_context}
    # question_context = f"""
    # What are the expected human health and infectious disease impacts of agroforestry interventions proposed in the Tanga Watershed Investment Program? 
    # Additionally, can these interventions be ranked based on their relative effect sizes on different human health and infectious disease outcomes? 
    # """

    # Context: {context_text}
    # Question: {query}

    response = client.chat.completions.create(
        model="Qwen/Qwen2.5-7B-Instruct-Turbo",
        messages=[
            {"role": "system", "content": "You are an AI assistant. Be descriptive and helpful."},
            {"role": "user", "content": prompt},
        ]
    )

    return response.choices[0].message.content


In [15]:
# Example query
query = "Can you identify and rank potential environmental levers for improved health outcomes?"
response = generate_rag_response(query)
print("AI Response:", response)

AI Response: Based on the provided context, several environmental levers can be identified as potential factors for improved health outcomes in low-income and middle-income settings where tree-based solutions are implemented. Here is a ranked list of these levers:

1. **Agricultural Yields**: The review found a significant positive effect on agricultural yields (standardized mean difference 0.41 [95% CI 0.11 to 0.70]). Improved agricultural productivity can lead to better nutrition, economic stability, and overall health.

2. **Dietary Diversity**: Enhanced dietary diversity (standardized mean difference 0.10 [95% CI 0.02 to 0.18]) is crucial for nutritional health, particularly in areas where food security is a challenge.

3. **Total Household Income**: Increased household income (standardized mean difference 0.21 [95% CI 0.09 to 0.33]) can improve access to healthcare, education, and other essential services, contributing to better health outcomes.

4. **Poverty Reduction**: Reductio

In [16]:
query = "What are the most important impacts of tree-based interventions on health and wellbeing?"
response = generate_rag_response(query)
print("AI Response:", response)

AI Response: Based on the provided context, the most important impacts of tree-based interventions on health and wellbeing in rural low-income and middle-income settings, as highlighted in the systematic review and meta-analysis, include:

1. **Improved Air Quality**: Trees can help reduce air pollution by absorbing pollutants and particulate matter, which can lead to better respiratory health for local communities.

2. **Increased Access to Medicinal Plants**: Tree-based interventions often include the planting of various tree species, some of which are used for traditional medicine. This can improve access to medicinal resources, enhancing health outcomes.

3. **Enhanced Water Quality and Availability**: Trees play a crucial role in water management by reducing soil erosion and improving water retention. This can lead to cleaner water sources and increased water availability, which are vital for health and sanitation.

4. **Mental Health Benefits**: Exposure to green spaces and trees

In [17]:
query = "How do different tree intervention practices impact food security?"
response = generate_rag_response(query)
print("AI Response:", response)

AI Response: Based on the provided context, different tree intervention practices impact food security in varying ways:

1. **Trees in Croplands**: This intervention had a significant positive effect on food security. The subgroup analysis showed a moderate effect size (0.35 [0.04 to 0.67]; p=0.025) with high heterogeneity (I²=95.2%).

2. **Payment for Ecosystem Services (PES)**: Similar to trees in croplands, PES interventions also had a significant positive effect on food security. The subgroup analysis indicated a positive impact, though the exact effect size was not provided in the given context.

3. **Protected Areas and Collaborative Forest Management**: These interventions did not have a significant positive effect on food security. The subgroup analysis found that these types of interventions had no detectable effects on food security.

In summary, tree-based interventions like planting trees in croplands and implementing Payment for Ecosystem Services (PES) showed promising re

In [18]:
query = "How do different tree intervention practices impact total income?"
response = generate_rag_response(query)
print("AI Response:", response)

AI Response: Based on the provided context, the study does not directly address how different tree intervention practices impact total income. The context mentions a forest plot analysis of the impact of interventions on income, but it does not specify the nature of these interventions or their effects on total income. The study appears to focus on the impact of tree-based interventions on health and wellbeing outcomes in rural low- and middle-income settings, with a meta-analysis approach. To answer the specific question about the impact on total income, we would need more detailed information from the full study or additional context.


In [19]:
query = "What is the difference between the impact of trees in croplands and PES on food security?"
response = generate_rag_response(query)
print("AI Response:", response)

AI Response: Based on the provided context, there isn't enough information to directly answer the question about the difference between the impact of trees in croplands and Payment for Ecosystem Services (PES) on food security. The context only mentions "Trees in croplands" three times, without providing any details about their impact or how they relate to food security or PES.

To properly answer this question, we would need additional information about:
1. The specific impacts of trees in croplands on food security
2. What Payment for Ecosystem Services (PES) is and its relation to food security
3. How these two concepts compare in terms of their effects on food security

Without this context, it's not possible to accurately describe the differences between the two. If you have more information or context about these topics, please provide it, and I'd be happy to help you formulate a more detailed answer.


In [25]:
query = f"""
    What are the expected human health and infectious disease impacts of agroforestry interventions proposed in the Tanga Watershed Investment Program? 
    Additionally, can these interventions be ranked based on their relative effect sizes on different human health and infectious disease outcomes? 
"""
retrieved_context = retrieve_context(query)
print("Relevant Context:", retrieved_context)

Relevant Context: ['25 Castle SE, Miller DC, Ordonez PJ, Baylis K, Hughes K. The impacts of agroforestry interventions on agricultural productivity, ecosystem services, and human wellbeing in low and middleincome countries: a systematic review. Campbell Syst Rev 2021; 17: e1167.', 'The variation in effect sizes by intervention type, overall programme aims, and study location suggests that the outcomes of interventions are highly contextualised. In general, interventions that emphasised livelihood improvement and community development, such as trees in croplands and PES, had positive impacts on the outcomes examined whereas interventions that targeted biodiversity or the removal of greenhouse gases had unfavourable or no detectable effect on agricultural yields, food security, total household income, and wellbeing. Income was the most studied outcome, which is not surprising given the povertyalleviation objectives of solutions implemented in LMIC settings; however,', 'When data allowed,

In [28]:
for t in retrieved_context:
    print(t)

# print(retrieved_context)

25 Castle SE, Miller DC, Ordonez PJ, Baylis K, Hughes K. The impacts of agroforestry interventions on agricultural productivity, ecosystem services, and human wellbeing in low and middleincome countries: a systematic review. Campbell Syst Rev 2021; 17: e1167.
The variation in effect sizes by intervention type, overall programme aims, and study location suggests that the outcomes of interventions are highly contextualised. In general, interventions that emphasised livelihood improvement and community development, such as trees in croplands and PES, had positive impacts on the outcomes examined whereas interventions that targeted biodiversity or the removal of greenhouse gases had unfavourable or no detectable effect on agricultural yields, food security, total household income, and wellbeing. Income was the most studied outcome, which is not surprising given the povertyalleviation objectives of solutions implemented in LMIC settings; however,
When data allowed, we used metaanalysis to p